# Import all dependencies 
Apart from the standard numpy, pandas and tensorflow libraries, also download the BERT files which contains models which were pretriained models, which will be used for fine tuning in this kernel. 

Also, thanks to https://www.kaggle.com/taindow/bert-a-fine-tuning-example 

reference : https://github.com/google-research/bert

@article{devlin2018bert,
  title={BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding},
  author={Devlin, Jacob and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1810.04805},
  year={2018}
}

In [1]:
import os
print(os.listdir("../input"))
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
print('TensorFlow version: ', tf.__version__)
sys.path.insert(0, '../input/pretrained-bert-including-scripts/master/bert-master')
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from datetime import datetime
from run_classifier import *
import modeling
import run_classifier
import optimization
import tokenization
from tqdm import tqdm

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
start_time = datetime.now()

['jigsaw-unintended-bias-in-toxicity-classification', 'pretrained-bert-including-scripts']
TensorFlow version:  1.13.1


# Dataset
* Upload all the data
* Add a new column called labels - any target score > 0.5 is assigned label 1 else 0
* Use only 30% of data

In [2]:
train=pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

# Pre-processing text
* Split train set into train and evalaution sets
* Remove stopwords and limit the length of sentence to 72 words in train, dev and test sets
* Convert to .tsv files


In [3]:
def remove_Stopwords(pd_frame):
    stop_words = {'not', 'won', 'but', 'this', 'most', "isn't", 'have', 'just', 'themselves', 'too', 'isn', 'hadn', 'about', 'from', 'both', "don't", 'hasn', "mightn't", 'your', 'his', 'than', 'so', 'now', 'until', 'they', 'ain', 'does', 'itself', 'or', 'off', "aren't", 'haven', 'i', 'you', 'he', 'why', 'it', 'under', 'd', 'mightn', 'up', 'each', 'down', 'y', 'o', 're', 'wouldn', "should've", 'no', 'which', 'aren', 'a', "you'll", "mustn't", 'doing', "didn't", 'same', 't', 'whom', "shan't", 'an', 'don', "wasn't", 'its', 'those', 'own', 'yours', 'myself', 'and', 'has', 'wasn', 'll', "hasn't", 'was', 'in', 'few', 'other', "couldn't", 'then', 'be', 'being', 'nor', "needn't", 'can', "won't", 'couldn', 'weren', 'been', 'for', "shouldn't", 'there', 'needn', 'yourself', 'how', 'her', 'herself', 'below', "you're", 'when', 'very', "haven't", 'into', 'didn', 'them', 'to', 'above', 'shan', 'some', 'are', 'on', 'is', 'their', 'at', 'am', 'hers', 'doesn', 'between', 'while', 's', 'should', 'theirs', 'himself', "that'll", 'ours', 'yourselves', 'what', 'again', 'had', 'ma', 'our', "you'd", 'my', 'out', "she's", 'she', 'if', "weren't", 'that', 'these', 'will', 'with', 'against', 'do', 'ourselves', 'all', 'who', 'as', 'here', "wouldn't", 've', 'through', 'the', 'after', "hadn't", 'of', 'having', 'once', 'only', 'because', 'where', "it's", 'by', 'shouldn', "doesn't", 'we', 'during', 'over', 'any', "you've", 'him', 'me', 'more', 'did', 'further', 'such', 'm', 'mustn', 'before', 'were'}
    list_sent = []
    for i in tqdm(range(len(pd_frame))):
        new_str = ''
        tot_length = 70
        sent = pd_frame.iloc[i]
        for j in sent.split():
            if j not in stop_words:
                new_str += ' ' + j.lower()
        list_sent.append(new_str)
    new_pd = pd.DataFrame(list_sent)
    return list_sent

In [4]:
train['comment_text'] = train['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

test['comment_text'] = test['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

train['dummy_1'] = 'meh'
train['dummy_2'] = '*'
train = train[['dummy_1','target','dummy_2','comment_text']]
train['target'] = np.where(train['target']>=0.5,1,0)

train_set = train.sample(frac=0.5,random_state = 42)
train, dev, y_train, y_test = train_test_split(train_set, train_set['target'], test_size=0.33, random_state=42)
train = train_set
train['comment_text'] = remove_Stopwords(train['comment_text'])
dev['comment_text'] = remove_Stopwords(dev['comment_text'])
test['comment_text'] = remove_Stopwords(test['comment_text'])

print(datetime.now() - start_time)

100%|██████████| 297805/297805 [00:11<00:00, 26549.16it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 97320/97320 [00:03<00:00, 26062.04it/s]

0:01:32.331944


In [5]:
train.to_csv('train.tsv', sep='\t', index=False, header=False)
dev.to_csv('dev.tsv', sep='\t', index=False, header=False)
test.to_csv('test.tsv', sep='\t', index=False, header=True)

## Overview of train and test file


In [6]:
print(train.head())
print(dev.head())
print(test.head())

        dummy_1                        ...                                                               comment_text
286892      meh                        ...                           what breathe fresh air someone embraces commo...
419218      meh                        ...                           your jewish friends ones told zionists contro...
1055330     meh                        ...                           possible collusion trump affiliates debunked,...
1382764     meh                        ...                           exactly. we need % gdp spending cap federal l...
256049      meh                        ...                           by comment, even voted ndp pq, trudeau demons...

[5 rows x 4 columns]
        dummy_1                        ...                                                               comment_text
762659      meh                        ...                           "the author quite correct saying real economy...
303985      meh                   

# Convert data into BERT acceptable format
 Bert has a processor to convert comment_text and corresponding target value into the InputExample class(with attributes: text_a, labels, guid). In order to use BERT we need to convert our input training examples into InputExample class method

In [7]:
class FLAGS(object):
    BERT_BASE_DIR='../input/pretrained-bert-including-scripts/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
    vocab_file=BERT_BASE_DIR+'vocab.txt'
    bert_config_file=BERT_BASE_DIR+'bert_config.json'
    init_checkpoint=BERT_BASE_DIR+'bert_model.ckpt'
    output_dir = "./"
    do_lower_case = True
    max_seq_length = 72

## ColaProcessor

In [8]:
processor = ColaProcessor()
label_list = processor.get_labels()
tokenizer = tokenization.FullTokenizer(vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

# Hyperparameters

In [9]:
BATCH_SIZE = 32
MAX_SEQ_LENGTH = 72
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_SUMMARY_STEPS = 100
tpu_cluster_resolver = None
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
use_tpu = False
master = None 
save_checkpoints_steps = 99999999
iterations_per_loop = 1000
num_tpu_cores = 8
tpu_cluster_resolver = None
output_dir = './'
data_dir = './'


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



# Extracting features from 'comment_text'
Extract features form comment_text for train,test and dev files and convert into a tf_record file

In [10]:

train_examples = processor.get_train_examples('./')
train_file = os.path.join(output_dir, "train.tf_record")
file_based_convert_examples_to_features(train_examples, label_list,
                                        MAX_SEQ_LENGTH, tokenizer,
                                        train_file)
dev_examples = processor.get_dev_examples('./')
dev_file = os.path.join(output_dir, "dev.tf_record")
file_based_convert_examples_to_features(dev_examples, label_list,
                                        MAX_SEQ_LENGTH, tokenizer,
                                        dev_file)
predict_examples = processor.get_test_examples('./')
predict_file = os.path.join(output_dir, "predict.tf_record")
file_based_convert_examples_to_features(predict_examples, label_list,
                                        MAX_SEQ_LENGTH, tokenizer,
                                        predict_file)
print(datetime.now() - start_time)

0:26:07.719760


In [11]:
run_config = tf.contrib.tpu.RunConfig(cluster=tpu_cluster_resolver, master=master, model_dir=FLAGS.output_dir, save_checkpoints_steps=save_checkpoints_steps,
                                      tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=iterations_per_loop, num_shards=num_tpu_cores,
                                      per_host_input_for_training=is_per_host))

# Training

In [12]:
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

run_config = tf.contrib.tpu.RunConfig(
  cluster=tpu_cluster_resolver,
  master=master,
  model_dir=FLAGS.output_dir,
  save_checkpoints_steps=save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=iterations_per_loop,
      num_shards=num_tpu_cores,
      per_host_input_for_training=is_per_host))

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=FLAGS.init_checkpoint,
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=use_tpu,
      use_one_hot_embeddings=use_tpu)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=32)
      

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
    
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print(datetime.now() - start_time)


2:43:23.478672


In [13]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    
    #batch_size = params["batch_size"]
    batch_size = 64 # <----- hardcoded batch_size added here 
    
    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

# Evaluation - (optional)

dev_examples = processor.get_dev_examples(data_dir)
num_actual_dev_examples = len(dev_examples)
dev_drop_remainder = False
dev_batch_size = 32
tf.logging.info("***** Running evalua`tion*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(dev_examples), num_actual_dev_examples,
                len(dev_examples) - num_actual_dev_examples)
tf.logging.info("  Batch size = %d", dev_batch_size)

dev_drop_remainder = True if use_tpu else False
dev_input_fn = file_based_input_fn_builder(
    input_file=dev_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=dev_drop_remainder)

estimator.evaluate(input_fn=dev_input_fn)


In [14]:
dev_examples = processor.get_dev_examples(data_dir)
num_actual_dev_examples = len(dev_examples)
dev_drop_remainder = False
dev_batch_size = 32
tf.logging.info("***** Running evalua`tion*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(dev_examples), num_actual_dev_examples,
                len(dev_examples) - num_actual_dev_examples)
tf.logging.info("  Batch size = %d", dev_batch_size)

dev_drop_remainder = True if use_tpu else False
dev_input_fn = file_based_input_fn_builder(
    input_file=dev_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=dev_drop_remainder)

estimator.evaluate(input_fn=dev_input_fn)


{'eval_accuracy': 0.95725054,
 'eval_loss': 0.11092799,
 'loss': 0.11091082,
 'global_step': 28201}

# Prediction

In [15]:
predict_examples = processor.get_test_examples(data_dir)
num_actual_predict_examples = len(predict_examples)
predict_batch_size = 32
predict_drop_remainder = True

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", predict_batch_size)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=predict_drop_remainder)

result = estimator.predict(input_fn=predict_input_fn)

output_predict_file = os.path.join(output_dir, "test_results.tsv")

with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\t".join(
            str(class_probability)
            for class_probability in probabilities) + "\n"
        writer.write(output_line)
        num_written_lines += 1

In [16]:
sample_submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
predictions = pd.read_csv('./test_results.tsv', header=None, sep='\t')

submission = pd.concat([sample_submission.iloc[:,0], predictions.iloc[:,1]], axis=1)
submission.columns = ['id','prediction']
submission.to_csv('submission.csv', index=False, header=True)

In [17]:
submission.head()
print(datetime.now() - start_time)

3:00:28.952353
